In [ ]:
%pip install vanna

In [ ]:
import vanna
from vanna.remote import VannaDefault
from vanna.flask import VannaFlaskApp

In [ ]:
api_key = vanna.get_api_key('dinhvietquyen18@gmail.com')

vanna_model_name = "real_estate_model"
vn = VannaDefault(model=vanna_model_name, api_key=api_key)

In [ ]:
# DDL statements are powerful because they specify table names, colume names, types, and potentially relationships
vn.train(ddl="""
    CREATE TABLE IF NOT EXISTS dim_broker (
    broker_id      SERIAL PRIMARY KEY,
    update_date    DATE NOT NULL,
    broker_name    TEXT,
    broker_rank    TEXT
)
""")
vn.train(ddl="""
    CREATE TABLE IF NOT EXISTS dim_post (
    post_id        SERIAL PRIMARY KEY,
    update_date    DATE NOT NULL,
    title          TEXT,
    url            TEXT,
    posted_date    DATE,
    expired_date   DATE
)
""")
vn.train(ddl="""
    CREATE TABLE IF NOT EXISTS dim_project (
    project_id             SERIAL PRIMARY KEY,
    update_date            DATE NOT NULL,
    project_name           TEXT,
    investor               TEXT,
    project_area_range     TEXT,
    project_status         TEXT,
    address                TEXT,
    number_of_apartments   REAL,
    number_of_buildings    REAL
)
""")
vn.train(ddl="""
    CREATE TABLE IF NOT EXISTS fact_all_apartment (
    post_id               INT REFERENCES dim_post(post_id),
    update_date           DATE NOT NULL,
    broker_id             INT REFERENCES dim_broker(broker_id),
    project_id            INT REFERENCES dim_project(project_id),
    price                 FLOAT,
    price_unit            TEXT,
    price_per_m²          FLOAT,
    price_per_m²_unit     TEXT,
    area                  FLOAT,
    area_unit             TEXT,
    bedroom               REAL,
    bathroom              REAL
)
""")
# Documentation is important to explain the meaning of each column and how to interpret the data
vn.train(documentation="Our business defines the price_unit as tỷ đồng (billion), " \
"price_per_m²_unit as triệu đồng/m² (million VND per square meter), " \
"area_unit as m² (square meter)," \
"and bedroom/bathroom as the number of bedrooms/bathrooms respectively.")

vn.train(documentaion="price and price_per_m² both have null values, " \
"so we need to handle that in our SQL queries." \
"If any question about price or price_per_m², " \
"we will not return null values for those columns.")

In [ ]:
# Let's connect to the database
vn.connect_to_postgres(host='localhost', dbname='mydb', user='admin', password='admin', port='5432')

In [ ]:
# The information schema query may need some tweaking depending on your database. This is a good starting point.
df_information_schema = vn.run_sql("SELECT * FROM INFORMATION_SCHEMA.COLUMNS")

# This will break up the information schema into bite-sized chunks that can be referenced by the LLM
plan = vn.get_training_plan_generic(df_information_schema)
plan

# Run it to train
vn.train(plan=plan)

In [5]:
# Now we can run the Vanna Flask app to interact with our model
# This will start a web server that you can access in your browser
app = VannaFlaskApp(vn)
app.run()